<a href="https://colab.research.google.com/github/bieri2/ATMS597-Project2-GroupA/blob/master/groupb_project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [0]:
# Define function to download data from NCEI
# Function adapted from Stefanie Moline: https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/

def make_request(endpoint, payload=None):
    """
    Make a request to a specific endpoint on the weather API
    passing headers and optional payload.
    
    Parameters:
        - endpoint: The endpoint of the API you want to 
                    make a GET request to.
        - payload: A dictionary of data to pass along 
                   with the request.
    
    Returns:
        Response object.
    """
    return requests.get(
        f'https://www.ncdc.noaa.gov/cdo-web/api/v2/{endpoint}',
        headers={
            'token': 'fqMFYJqPpWnQIARJzXBQhWPbYEtJwpoH'
        },
        params=payload
    )

In [0]:
current = datetime.date(2017,12, 1)
end = datetime.date(2017, 12, 3)

results = []

while current < end:

    response = make_request(
        'data', 
        {
            'datasetid' : 'GHCND', # Global Historical Climatology Network - Daily (GHCND) dataset
            'stationid' : 'GHCND:ASN00008051', # Somewhere in Australia 
            'datatypeid': 'TMAX', # Request only TMAX
            'startdate' : current,
            'enddate' : current,
            'units' : 'metric'
        }
    )

    if response.ok:
        # we extend the list instead of appending to avoid getting a nested list
        results.extend(response.json()['results'][0])

    # update the current date to avoid an infinite loop
    current += datetime.timedelta(days=1)

In [60]:
results

[{'attributes': ',,S,',
  'datatype': 'TMAX',
  'date': '2017-12-01T00:00:00',
  'station': 'GHCND:ASN00008051',
  'value': 34.0},
 {'attributes': ',,S,',
  'datatype': 'TMAX',
  'date': '2017-12-02T00:00:00',
  'station': 'GHCND:ASN00008051',
  'value': 41.0}]